# Original static topics

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import re, json, numpy as np

## Load preprocessed data

Data is already gathered in a single parquet files with selected columns about opinions (proposals):
- title and descriptions
- votes (number, positive, mitigate, negative)
- arguments (pros, cons)

In [24]:
df = pd.read_parquet('leVraiDebat-opinions.parquet')

In [25]:
df.columns

Index(['master_tag', 'contributions_id', 'contribution_versions_title',
       'contribution_versions_bodyText', 'contributions_votesCount',
       'contributions_votesCountOk', 'contributions_votesCountMitige',
       'contributions_votesCountNok', 'contributions_argumentsCountFor',
       'contributions_argumentsCountAgainst'],
      dtype='object')

# Selection of opinions

For each topic save the 20 best proposals

In [26]:
#df.set_index('contributions_id', inplace=True)

In [27]:
topics = df.master_tag.unique()
topics

array(['democratie-institutions-referendum-dinitiative-citoyenne',
       'economie-finances-travail-compte-public',
       'education-jeunesse-enseignement-superieur-recherche-et-innovation',
       'europe-affaires-etrangeres-outre-mer', 'justice-police-armee',
       'sante-solidarite-handicap', 'sport-culture', 'expression-libre',
       'transition-ecologique-solidaire-agriculture-alimentation'],
      dtype=object)

In [30]:
saved_cols = ['contributions_id', 'contribution_versions_title',
       'contribution_versions_bodyText', 'contributions_votesCount',
       'contributions_votesCountOk', 'contributions_votesCountMitige',
       'contributions_votesCountNok', 'contributions_argumentsCountFor',
       'contributions_argumentsCountAgainst']
for i, topic in enumerate(topics):
    main_contributions = df[df.master_tag == topic].sort_values('contributions_votesCountOk', ascending=False)[:20]
    main_contributions[saved_cols].to_json('topic_%d_main_contributions.json' % i, orient='records')

In [31]:
# Without topic filter => root
main_contributions = df.sort_values('contributions_votesCountOk', ascending=False)[:20]
main_contributions[saved_cols].to_json('topic_root_main_contributions.json', orient='records')